In [1]:
import torch.nn as nn
from collections import OrderedDict
from torch.optim import SGD
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs #---Builds a synthetic dataset of example data
import torch

In [2]:
# construction a shallow sequential neural network


def get_model(inFeatures=4, hiddenDim=8, nbClasses=3):
    mlpModel = nn.Sequential(OrderedDict([
        ('hidden_layer_1',nn.Linear(inFeatures,hiddenDim)),
        ('activation_1',nn.ReLU()),
        ('output_layer',nn.Linear(hiddenDim,nbClasses))
    ]))
    
    return mlpModel
mlp = get_model()
print(mlp)



Sequential(
  (hidden_layer_1): Linear(in_features=4, out_features=8, bias=True)
  (activation_1): ReLU()
  (output_layer): Linear(in_features=8, out_features=3, bias=True)
)


In [3]:
def next_batch(inputs,targets,batchSize):
    
#     ```
#     inputs: Our input data to the neural network
#     targets: Our target output values (i.e., what we want our neural network to accurately predict)
#     batchSize: Size of data batch
#     ```
    # loop over the datset in batchsize chunks
    for i in range(0,inputs.shape[0],batchSize):
        #yeild tuple of current batched data and labels
        yield(inputs[i:i + batchSize],targets[i:i + batchSize])

In [4]:
# HyPERPARAMETERS
BATCH_SIZE  = 64
EPOCHS = 30
LR = 1e-2

#determine the training device
DEVICE = 'Cuda' if torch.cuda.is_available() else 'cpu'
print("[INFO] training using {}....".format(DEVICE))


[INFO] training using cpu....


In [5]:
print("[INFO] preparing data....")
# generate a 3-class classification problem with 1000 data points,
(X,y) = make_blobs(n_samples=1000,n_features=4,centers=3,
                  cluster_std=2.5,random_state=95)

(trainX,testX,trainY,testY) = train_test_split(X,y,
                    test_size=0.15,random_state=95)

# the training and testing data is converted to PyTorch tensors from NumPy arrays
trainX = torch.from_numpy(trainX).float()
trainY = torch.from_numpy(trainY).float()
testY = torch.from_numpy(testY).float()
testX = torch.from_numpy(testX).float()

[INFO] preparing data....


In [6]:
#initialize our model 
mlp = mlp.to(DEVICE)

# loss function and optimizer
opt = SGD(mlp.parameters(),lr=LR)
lossFunc = nn.CrossEntropyLoss()


In [7]:
trainTemplate = "epoch: {} test loss: {.3f} test accuracy: {.3f}"

#loop through the epochs
for epoch in range(0,EPOCHS):
    # initialize tracker variables and set our model to trainable
    print("[INFO] epoch: {}...".format(epoch+1))
    trainLoss = 0
    trainAcc = 0
    samples = 0
    mlp.train()
    # loop over the current data batch
    for (batchX,batchY) in next_batch(trainX,trainY,BATCH_SIZE):
        
        # move current batch to DEVICE, run through model calc loss
        (batchX,batchY) = (batchX.to(DEVICE), batchY.to(DEVICE))
        predictions = mlp(batchX)
        loss = lossFunc(predictions, batchY.long())
        
        #set gradients accumulated from prev step to zero
        #perform back propagation and update model params
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        #update train loss, accuracy, and num of samples visited
        trainLoss += loss.item() * batchY.size(0)
        trainAcc +=  (predictions.max(1)[1] == batchY).sum().item()
        samples += batchY.size(0)
        
        
    trainTemplate = "epoch: {} train loss: {:.3f} train accuracy: {:.3f}"
    print(trainTemplate.format(epoch+1,(trainLoss/samples),(trainAcc /samples)))
    
    # initialize tracker variables
    testLoss = 0
    testAcc = 0
    samples = 0
    mlp.eval()

    # intialize with no gradients
    with torch.no_grad():
        for (batchX,batchY) in next_batch(testX,testY,BATCH_SIZE):

            # move current batch to DEVICE, run through model calc loss
            (batchX,batchY) = (batchX.to(DEVICE), batchY.to(DEVICE))
            predictions = mlp(batchX)
            loss = lossFunc(predictions, batchY.long())


            #update train loss, accuracy, and num of samples visited
            testLoss += loss.item() * batchY.size(0)
            testAcc +=  (predictions.max(1)[1] == batchY).sum().item()
            samples += batchY.size(0)

        testTemplate = "epoch: {} test loss: {:.3f} test accuracy: {:.3f}"
        print(testTemplate.format(epoch+1,(testLoss/samples),(testAcc /samples)))
        print("")

[INFO] epoch: 1...
epoch: 1 train loss: 1.265 train accuracy: 0.327
epoch: 1 test loss: 0.863 test accuracy: 0.573

[INFO] epoch: 2...
epoch: 2 train loss: 0.801 train accuracy: 0.638
epoch: 2 test loss: 0.622 test accuracy: 0.720

[INFO] epoch: 3...
epoch: 3 train loss: 0.616 train accuracy: 0.680
epoch: 3 test loss: 0.505 test accuracy: 0.740

[INFO] epoch: 4...
epoch: 4 train loss: 0.516 train accuracy: 0.754
epoch: 4 test loss: 0.434 test accuracy: 0.813

[INFO] epoch: 5...
epoch: 5 train loss: 0.449 train accuracy: 0.812
epoch: 5 test loss: 0.382 test accuracy: 0.853

[INFO] epoch: 6...
epoch: 6 train loss: 0.395 train accuracy: 0.865
epoch: 6 test loss: 0.338 test accuracy: 0.880

[INFO] epoch: 7...
epoch: 7 train loss: 0.346 train accuracy: 0.895
epoch: 7 test loss: 0.297 test accuracy: 0.907

[INFO] epoch: 8...
epoch: 8 train loss: 0.301 train accuracy: 0.920
epoch: 8 test loss: 0.257 test accuracy: 0.927

[INFO] epoch: 9...
epoch: 9 train loss: 0.260 train accuracy: 0.939
epoc